In [4]:
from settrade_v2 import Investor
from pprint import pprint
from settrade_v2 import Investor
import psycopg2
import pandas as pd
import time
from psycopg2 import sql
from datetime import datetime
from pprint import pprint
import mplfinance as mpf
import matplotlib.pyplot as plt
import numpy as np
from cassandra.cluster import Cluster

In [5]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [16]:
investor = Investor(
                app_id="iOk7g707jmQiJMj0",                                 
                app_secret="ALnKkLC6kyej4gykWT6X8MT9AyR3bsqs+7iVeKZrHu6v", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="babi-D")            

account_info = deri.get_account_info()

account_info       

{'creditLine': 10000000.0,
 'excessEquity': 10000000.0,
 'cashBalance': 10000000.0,
 'equity': 10000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 10000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [29]:
# กรณี Investor
market = investor.MarketData()

In [30]:
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
pprint(res)

{'aumSize': None,
 'average': None,
 'change': None,
 'eps': 0.99837,
 'exchange': None,
 'exercisePrice': None,
 'exerciseRatio': None,
 'high': None,
 'impliedVolatility': None,
 'inav': None,
 'instrumentType': 'STOCK',
 'intrinsicValue': None,
 'last': None,
 'lastTradingDate': None,
 'low': None,
 'marketStatus': 'Close',
 'maturityDate': None,
 'moneyness': None,
 'pbv': 4.25,
 'pe': 29.1,
 'percentChange': None,
 'percentYield': 2.0899999141693115,
 'securityType': 'CS',
 'status': '',
 'symbol': 'AOT',
 'theoretical': None,
 'toLastTrade': None,
 'totalBuyVolume': 0,
 'totalNoSideVolume': 0,
 'totalSellVolume': 0,
 'totalVolume': 0,
 'underlying': None,
 'underlyingPrice': None}


In [20]:

print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 0.99837, 'pe': 29.1, 'pbv': 4.25, 'percentYield': 2.0899999141693115, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [21]:
session.set_keyspace('data_stock')

In [22]:
session.execute("""
CREATE TABLE IF NOT EXISTS financal_data_fromsettradeAPI (
    symbol TEXT PRIMARY KEY,
    aumSize DOUBLE,
    average DOUBLE,
    change DOUBLE,
    eps DOUBLE,
    exchange TEXT,
    exercisePrice DOUBLE,
    exerciseRatio DOUBLE,
    high DOUBLE,
    impliedVolatility DOUBLE,
    inav DOUBLE,
    instrumentType TEXT,
    intrinsicValue DOUBLE,
    last DOUBLE,
    lastTradingDate DATE,
    low DOUBLE,
    marketStatus TEXT,
    maturityDate DATE,
    moneyness TEXT,
    pbv DOUBLE,
    pe DOUBLE,
    percentChange DOUBLE,
    percentYield DOUBLE,
    securityType TEXT,
    status TEXT,
    theoretical DOUBLE,
    toLastTrade TEXT,
    totalVolume BIGINT,
    totalBuyVolume BIGINT,
    totalNoSideVolume BIGINT,
    totalSellVolume BIGINT,
    underlying TEXT,
    underlyingPrice DOUBLE
)
""")

In [49]:
# ✅ ฟังก์ชันกัน error field ที่ไม่ใช่ int/float/str
def safe_get(data, key, default=None):
    val = data.get(key, default)
    return val if isinstance(val, (int, float, str, type(None))) else default

# ✅ ฟังก์ชันดึงข้อมูลและ insert
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        market = investor.MarketData()
        res = market.get_quote_symbol(symbol)

        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # ✅ แปลงวันที่ (หากมี)
        def parse_date(timestamp):
            if isinstance(timestamp, (int, float)):
                return datetime.fromtimestamp(timestamp).date()
            return None

        session.execute("""
            INSERT INTO financal_data_fromsettradeAPI (
                symbol, aumSize, average, change, eps, exchange,
                exercisePrice, exerciseRatio, high, impliedVolatility,
                inav, instrumentType, intrinsicValue, last,
                lastTradingDate, low, marketStatus, maturityDate,
                moneyness, pbv, pe, percentChange, percentYield,
                securityType, status, theoretical, toLastTrade,
                totalVolume, totalBuyVolume, totalNoSideVolume,
                totalSellVolume, underlying, underlyingPrice
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                      %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                      %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                      %s, %s, %s)
        """, (
            symbol,
            safe_get(res, 'aumSize'),
            safe_get(res, 'average'),
            safe_get(res, 'change'),
            safe_get(res, 'eps'),
            safe_get(res, 'exchange'),
            safe_get(res, 'exercisePrice'),
            safe_get(res, 'exerciseRatio'),
            safe_get(res, 'high'),
            safe_get(res, 'impliedVolatility'),
            safe_get(res, 'inav'),
            safe_get(res, 'instrumentType'),
            safe_get(res, 'intrinsicValue'),
            safe_get(res, 'last'),
            parse_date(res.get('lastTradingDate')),
            safe_get(res, 'low'),
            safe_get(res, 'marketStatus'),
            parse_date(res.get('maturityDate')),
            safe_get(res, 'moneyness'),
            safe_get(res, 'pbv'),
            safe_get(res, 'pe'),
            safe_get(res, 'percentChange'),
            safe_get(res, 'percentYield'),
            safe_get(res, 'securityType'),
            safe_get(res, 'status'),
            safe_get(res, 'theoretical'),
            safe_get(res, 'toLastTrade'),
            safe_get(res, 'totalVolume'),
            safe_get(res, 'totalBuyVolume'),
            safe_get(res, 'totalNoSideVolume'),
            safe_get(res, 'totalSellVolume'),
            safe_get(res, 'underlying'),
            safe_get(res, 'underlyingPrice')
        ))

        print(f"✅ ข้อมูล {symbol} ถูกบันทึกเรียบร้อย")

    except Exception as e:
        print(f"❌ ERROR [{symbol}]: {e}")

# ✅ โหลดรายชื่อหุ้นจากไฟล์ Excel
try:
    symbols_df = pd.read_excel("get_stcok_name/stocth_names.xlsx")  # ต้องมีคอลัมน์ 'symbol'
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' → ใช้ชุดหุ้นตัวอย่าง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# 🔁 ดึงและบันทึกหุ้นแต่ละตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(1.5)  # ⏳ ป้องกัน rate limit

print("\n🎉 เสร็จสิ้นการโหลดและบันทึกข้อมูล Fundamental ทุกตัว!")

🔍 กำลังดึงข้อมูล Financial Data สำหรับ 24CS...
✅ ข้อมูล 24CS ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 2S...
✅ ข้อมูล 2S ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 3BBIF...
✅ ข้อมูล 3BBIF ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A...
✅ ข้อมูล A ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A5...
✅ ข้อมูล A5 ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAI...
✅ ข้อมูล AAI ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAV...
✅ ข้อมูล AAV ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ABM...
✅ ข้อมูล ABM ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACAP...
✅ ข้อมูล ACAP ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACC...
✅ ข้อมูล ACC ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACE...
✅ ข้อมูล ACE ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACG...
✅ ข้อมูล ACG ถูกบันทึกเรียบร้อย
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ADB...
✅ ข้อมูล ADB ถูกบั